In [1]:
import torch
import numpy as np
import pandas as pd
import pickle
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split


In [11]:
device = torch.device("cuda")  # torch.device('cpu')

In [49]:
obj = pd.read_pickle(r'train.pkl')
max_len = max(len(arr) for arr, _ in obj)

In [57]:
padded_data = []
labels = []

for arr, label in obj:
    padded = np.pad(arr, (0, max_len - len(arr)), constant_values=0)
    padded_data.append(padded)
    labels.append(label)

# Create DataFrame
df = pd.DataFrame(padded_data)
df['label'] = labels

print(df.head())

       0      1      2      3      4      5      6      7      8      9  ...  \
0   -1.0   -1.0   -1.0   -1.0  144.0  144.0  144.0   64.0   67.0    0.0  ...   
1   -1.0   -1.0  144.0  144.0   34.0    0.0    4.0    2.0   65.0  144.0  ...   
2   66.0  100.0  148.0  148.0  146.0   64.0  146.0  148.0   82.0    0.0  ...   
3  147.0   65.0   64.0  146.0    8.0   50.0  160.0   50.0  100.0   66.0  ...   
4  144.0  144.0  144.0  144.0  144.0   64.0   64.0   64.0    0.0  112.0  ...   

   6299  6300  6301  6302  6303  6304  6305  6306  6307  label  
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0      0  
1   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0      0  
2   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0      0  
3   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0      0  
4   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0      0  

[5 rows x 6309 columns]


In [58]:
classes = df['label'].values
data = df.values[:, :-1]

In [59]:
np.unique(classes).shape
data = torch.from_numpy(data).float()
data_targets = torch.from_numpy(classes).long()
rng = np.random.default_rng(73512)

train_indices = rng.random(len(data)) > 0.3
test_indices = ~train_indices
train_set = torch.utils.data.TensorDataset(
    data[train_indices], data_targets[train_indices]
)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=32)
test_data, test_targets = data[test_indices], data_targets[test_indices]

In [62]:
import torch.nn as nn
class LSTMRegressor(nn.Module):
    def __init__(
        self, input_size, hidden_size, num_layers, out_size, bidirectional=False
    ):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        if bidirectional:
            self.bidirectional = 2
        else:
            self.bidirectional = 1
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            bidirectional=bidirectional,
        )
        self.fc = nn.Linear(hidden_size *6308 * self.bidirectional, out_size)

    def init_hidden(self, batch_size):
        hidden = torch.zeros(
            self.num_layers * self.bidirectional, batch_size, self.hidden_size
        )
        state = torch.zeros(
            self.num_layers * self.bidirectional, batch_size, self.hidden_size
        )
        return hidden, state

    def forward(self, x, hidden):
        x = torch.transpose(x, 0, 1)
        all_outputs, hidden = self.lstm(x, hidden)
        all_outputs = torch.transpose(all_outputs, 0, 1)
        out = torch.flatten(all_outputs, 1)
        x = self.fc(out)
        return x, hidden


model = LSTMRegressor(1, 5, 2, 5, bidirectional=True).to(device)
model

LSTMRegressor(
  (lstm): LSTM(1, 5, num_layers=2, bidirectional=True)
  (fc): Linear(in_features=63080, out_features=5, bias=True)
)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fun = nn.CrossEntropyLoss()

# Training loop
for epoch in range(101):
    for x, targets in train_loader:
        x = x.to(device).unsqueeze(2)
        #         x = x.unsqueeze(2)
        targets = targets.to(device)
        hidden, state = model.init_hidden(x.size(0))
        hidden, state = hidden.to(device), state.to(device)
        preds, _ = model(x, (hidden, state))
        preds = preds.squeeze(1)
        optimizer.zero_grad()
        loss = loss_fun(preds, targets) 
        loss.backward()
        optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch: {epoch}, loss: {loss.item():.3}")

Epoch: 0, loss: 49.0
Epoch: 10, loss: 1.71
Epoch: 20, loss: 2.22
Epoch: 30, loss: 1.95
Epoch: 40, loss: 1.12
Epoch: 50, loss: 0.793
Epoch: 60, loss: 0.469
Epoch: 70, loss: 0.3
Epoch: 80, loss: 0.258
Epoch: 90, loss: 0.148
Epoch: 100, loss: 0.0827


In [64]:
with torch.no_grad():
    hidden, state = model.init_hidden(len(test_data))
    hidden, state = hidden.to(device), state.to(device)
    preds, _ = model(test_data.to(device).unsqueeze(2), (hidden, state))
print(
    f"Accuracy: {(torch.argmax(preds, 1).cpu() == test_targets).sum().item() / len(test_targets):.3}"
)

Accuracy: 0.608


In [65]:
torch.save(model.state_dict(), "model_rekurencyjny_1.pt")

In [68]:
the_model = LSTMRegressor(1, 5, 2, 5, bidirectional=True).to(device)
the_model.load_state_dict(torch.load("model_rekurencyjny_1.pt"))

<All keys matched successfully>

In [69]:
with torch.no_grad():
    hidden, state = model.init_hidden(len(test_data))
    hidden, state = hidden.to(device), state.to(device)
    preds, _ = model(test_data.to(device).unsqueeze(2), (hidden, state))
print(
    f"Accuracy: {(torch.argmax(preds, 1).cpu() == test_targets).sum().item() / len(test_targets):.3}"
)

Accuracy: 0.608
